In [64]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.parsing.preprocessing import strip_short, remove_stopwords, strip_non_alphanum, strip_numeric, strip_multiple_whitespaces, strip_punctuation, strip_tags
import pickle
%matplotlib inline

In [ ]:
# Download data
#import requests
#request = requests.get("https://drive.google.com/uc?export=download&id=1wHt8PsMLsfX5yNSqrt2fSTcb8LEiclcf")
#with open("data.zip", "wb") as file:
#    file.write(request.content)

# Unzip data
#import zipfile
#with zipfile.ZipFile('data.zip') as zip:
#    zip.extractall('data')

In [ ]:
 # Load data and set labels
#data_complaint = pd.read_csv('data/complaint1700.csv')
#data_complaint['label'] = 0
#data_non_complaint = pd.read_csv('data/noncomplaint1700.csv')
#data_non_complaint['label'] = 1

# Concatenate complaining and non-complaining data
#data = pd.concat([data_complaint, data_non_complaint], axis=0).reset_index(drop=True)

# Drop 'airline' column
#data.drop(['airline'], inplace=True, axis=1)

# Display 5 random samples
#data.sample(5)

# LOAD TWEET DATA

In [65]:
#data = pd.read_csv('data/tweets.csv')
data = pd.read_csv('data/auto_tweets.csv')

data = data.sample(frac = 1) 


In [66]:
data['auto_sentiment'] = data['auto_sentiment'].map(lambda x: x+1)

In [67]:
test_data = data[:10000]
data = data[10000:]

test_data = test_data[:10000]
data = data[:100000]

In [68]:
print(len(data))
print(len(test_data))

100000
10000


In [69]:
#data= data[:1000]
data.sample(10)


,text,sentiment,auto_sentiment
1440517,20 min. after twelve...just finished making a ...,1,1
1235950,......never drive faSter than yOur guardian an...,1,1
1035855,"@RowdyKittens Well, a tiny piece of the world...",1,2
1211992,Had a long day n now I'm trying 2 chill with t...,1,1
896863,@angelicaeff WELL HEY HEY BABY ITS NEVER TOO L...,1,0
550683,Road trip ! To birmingham wat a shit whole !,0,0
1132065,My sister got the kids &amp; Up. I got my dad ...,1,1
1578880,@kestrelwines yay! so glad to see u on twitter...,1,2
517324,"@Live_for_Films Oh, bum. I love that book. Ho...",0,2
208642,My day officially sucks now! Sure wish I could...,0,2


In [70]:
from sklearn.model_selection import train_test_split

X = data.text.values
y = data.auto_sentiment.values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=2020)

In [71]:
# Load test data
#test_data = pd.read_csv('data/test_data.csv')

# Keep important columns
#test_data = test_data[['id', 'tweet']]

# Display 5 samples from the test data
test_data.sample(5)


,text,sentiment,auto_sentiment
817319,Pursuing a new business venture. Nervous and e...,1,2
1089648,@Lumiel Aren't you sweet?,1,0
978914,@blissmanu I started at 15:50 and reached the ...,1,2
212878,@alexandramusic Diversity won! damnn i wanted ...,0,2
1099452,@nofe do you really want to spend 4 months doi...,1,2


In [72]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: GeForce GTX 1080 Ti


In [73]:
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

#def text_preprocessing(s):
#    """
#    - Lowercase the sentence
#    - Change "'t" to "not"
#    - Remove "@name"
#    - Isolate and remove punctuations except "?"
#    - Remove other special characters
#    - Remove stop words except "not" and "can"
#    - Remove trailing whitespace
#    """
#    s = s.lower()
#    # Change 't to 'not'
#    s = re.sub(r"\'t", " not", s)
#    # Remove @name
#    s = re.sub(r'(@.*?)[\s]', ' ', s)
#    # Isolate and remove punctuations except '?'
#    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
#    s = re.sub(r'[^\w\s\?]', ' ', s)
#    # Remove some special characters
#    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
#    # Remove stopwords except 'not' and 'can'
#    s = " ".join([word for word in s.split()
#                  if word not in stopwords.words('english')
#                  or word in ['not', 'can']])
#    # Remove trailing whitespace
#    s = re.sub(r'\s+', ' ', s).strip()
    
#    return s


def text_preprocessing(text):
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    text = strip_tags(text)
    text = strip_non_alphanum(text)
    text = strip_punctuation(text)
    text = strip_numeric(text)
    #text = remove_stopwords(text)
    text = text.lower()
    text = strip_multiple_whitespaces(text)
    #text = strip_short(text, minsize = 3)

    return text


[nltk_data] Downloading package stopwords to /home/felix/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [74]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess text
X_train_preprocessed = np.array([text_preprocessing(text) for text in X_train])
X_val_preprocessed = np.array([text_preprocessing(text) for text in X_val])

# Calculate TF-IDF
#tf_idf = TfidfVectorizer(ngram_range=(1, 3),
#                         binary=True,
#                         smooth_idf=False)
#X_train_tfidf = tf_idf.fit_transform(X_train_preprocessed)
#X_val_tfidf = tf_idf.transform(X_val_preprocessed)

CPU times: user 1.74 s, sys: 28 ms, total: 1.76 s
Wall time: 1.76 s


In [75]:
X_train[38:39]

array(['got my tickets for the Blink 182 irvine show  be jealous.'],
      dtype=object)

In [76]:
X_train_preprocessed[38]

'got my tickets for the blink irvine show be jealous '

In [77]:
#preprocessing_for_bert([X_train_preprocessed[38]])

In [78]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

def get_auc_CV(model):
    """
    Return the average AUC score from cross-validation.
    """
    # Set KFold to shuffle data before the split
    kf = StratifiedKFold(5, shuffle=True, random_state=1)

    # Get AUC scores
    auc = cross_val_score(
        model, X_train_tfidf, y_train, scoring="roc_auc", cv=kf)

    return auc.mean()

In [79]:
#from sklearn.naive_bayes import MultinomialNB

#res = pd.Series([get_auc_CV(MultinomialNB(i))
#                 for i in np.arange(1, 10, 0.1)],
#                index=np.arange(1, 10, 0.1))

#best_alpha = np.round(res.idxmax(), 2)
#print('Best alpha: ', best_alpha)

#plt.plot(res)
#plt.title('AUC vs. Alpha')
#plt.xlabel('Alpha')
#plt.ylabel('AUC')
#plt.show()

In [80]:
from sklearn.metrics import accuracy_score, roc_curve, auc

def evaluate_roc(probs, y_true):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """
    preds = probs[:, 1]
    fpr, tpr, threshold = roc_curve(y_true, preds)
    roc_auc = auc(fpr, tpr)
    print(f'AUC: {roc_auc:.4f}')
       
    # Get accuracy over the test set
    y_pred = np.where(preds >= 0.5, 1, 0)
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy*100:.2f}%')
    
    # Plot ROC AUC
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [81]:
# Compute predicted probabilities
#nb_model = MultinomialNB(alpha=1.8)
#nb_model.fit(X_train_tfidf, y_train)
#probs = nb_model.predict_proba(X_val_tfidf)

# Evaluate the classifier
#evaluate_roc(probs, y_val)

In [82]:
#def text_preprocessing(text):
#    """
#    - Remove entity mentions (eg. '@united')
#    - Correct errors (eg. '&amp;' to '&')
#    @param    text (str): a string to be processed.
#    @return   text (Str): the processed string.
#    """
#    # Remove '@name'
#    text = re.sub(r'(@.*?)[\s]', ' ', text)
#
#    # Replace '&amp;' with '&'
#    text = re.sub(r'&amp;', '&', text)
#
#    # Remove trailing whitespace
#    text = re.sub(r'\s+', ' ', text).strip()

#    return text

#def text_preprocessing(text):
#    text = re.sub(r'(@.*?)[\s]', ' ', text)
#    text = strip_tags(text)
#    text = strip_non_alphanum(text)
#    text = strip_punctuation(text)
#    text = strip_numeric(text)
#    #text = remove_stopwords(text)
#    text = text.lower()
#    text = strip_multiple_whitespaces(text)
#    #text = strip_short(text, minsize = 3)
#
#    return text

In [83]:
data[:1]['text']

180565    No Windows 7 support for VMware View yet.  (VM...
Name: text, dtype: object

# BERT

In [84]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        #print("aaaa: ", tokenizer.convert_ids_to_tokens(encoded_sent.get('input_ids')))
        #print("sent: " , sent)
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [85]:
# Concatenate train data and test data
#all_tweets = np.concatenate([data.tweet.values, test_data.tweet.values])
all_tweets = np.concatenate([data.text.values, test_data.text.values])

# Encode our concatenated data
encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in all_tweets]

# Find the maximum length
max_len = max([len(sent) for sent in encoded_tweets]) #230
print('Max length: ', max_len)

Max length:  216


In [86]:
# Specify `MAX_LEN`
MAX_LEN = max_len

# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
print('Original: ', X[0])
print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)


Original:  No Windows 7 support for VMware View yet.  (VMwareInfo.com) http://bit.ly/YIsYA
Token IDs:  [101, 2053, 3645, 2490, 2005, 1058, 2213, 8059, 3193, 2664, 1058, 2213, 8059, 2378, 14876, 4012, 8299, 2978, 1048, 2100, 12316, 6508, 2050, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tokenizing data...


In [87]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [88]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 3

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: user 33 µs, sys: 0 ns, total: 33 µs
Wall time: 36.5 µs


In [89]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [90]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [91]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   1.018215   |     -      |     -     |   12.96  
   1    |   40    |   0.902504   |     -      |     -     |   11.61  
   1    |   60    |   0.717939   |     -      |     -     |   11.32  
   1    |   80    |   0.645551   |     -      |     -     |   11.40  
   1    |   100   |   0.583537   |     -      |     -     |   11.38  
   1    |   120   |   0.553276   |     -      |     -     |   11.39  
   1    |   140   |   0.520482   |     -      |     -     |   11.39  
   1    |   160   |   0.516909   |     -      |     -     |   11.57  
   1    |   180   |   0.482380   |     -      |     -     |   11.73  
   1    |   200   |   0.444277   |     -      |     -     |   11.48  
   1    |   220   |   0.464593   |     -      |     -     |   11.43  
   1    |   240   |   0.497613   |     -      |     -     |   11.52  


   1    |  2320   |   0.256011   |     -      |     -     |   11.67  
   1    |  2340   |   0.240992   |     -      |     -     |   11.63  
   1    |  2360   |   0.246192   |     -      |     -     |   11.59  
   1    |  2380   |   0.201653   |     -      |     -     |   11.58  
   1    |  2400   |   0.239893   |     -      |     -     |   11.65  
   1    |  2420   |   0.186047   |     -      |     -     |   11.96  
   1    |  2440   |   0.216591   |     -      |     -     |   11.56  
   1    |  2460   |   0.200852   |     -      |     -     |   11.51  
   1    |  2480   |   0.262165   |     -      |     -     |   11.70  
   1    |  2500   |   0.248213   |     -      |     -     |   11.52  
   1    |  2520   |   0.241637   |     -      |     -     |   11.57  
   1    |  2540   |   0.226213   |     -      |     -     |   11.59  
   1    |  2560   |   0.244370   |     -      |     -     |   11.51  
   1    |  2580   |   0.192568   |     -      |     -     |   11.56  
   1    |  2600   | 

   2    |  1740   |   0.158154   |     -      |     -     |   11.87  
   2    |  1760   |   0.143108   |     -      |     -     |   11.48  
   2    |  1780   |   0.126684   |     -      |     -     |   11.48  
   2    |  1800   |   0.177529   |     -      |     -     |   11.50  
   2    |  1820   |   0.218080   |     -      |     -     |   11.66  
   2    |  1840   |   0.122699   |     -      |     -     |   11.53  
   2    |  1860   |   0.173750   |     -      |     -     |   11.49  
   2    |  1880   |   0.189682   |     -      |     -     |   11.52  
   2    |  1900   |   0.145839   |     -      |     -     |   11.59  
   2    |  1920   |   0.163030   |     -      |     -     |   11.61  
   2    |  1940   |   0.189116   |     -      |     -     |   11.62  
   2    |  1960   |   0.124833   |     -      |     -     |   11.80  
   2    |  1980   |   0.152838   |     -      |     -     |   12.02  
   2    |  2000   |   0.099069   |     -      |     -     |   11.49  
   2    |  2020   | 

# Pickle DUMP

In [92]:
with open('./bert_classifier.pickle', 'wb') as handle:
    pickle.dump(bert_classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Pickle load

In [ ]:
with open('./bert_classifier.pickle', 'rb') as handle:
        bert_classifier = pickle.load(handle)

In [93]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [94]:
# Compute predicted probabilities on the test set
probs_val = bert_predict(bert_classifier, val_dataloader)

probs_train = bert_predict(bert_classifier, train_dataloader)



# Evaluate the Bert classifier
#evaluate_roc(probs, y_val)

In [95]:
correct = 0
for i , p in enumerate(probs_train):
    if np.argmax(p) == y_train[i]:
        correct +=1       
print("train acc: " , correct / len(y_train))


correct = 0
for i , p in enumerate(probs_val):
    if np.argmax(p) == y_val[i]:
        correct +=1       
print("val acc: " , correct / len(y_val))

train acc:  0.36054444444444445
val acc:  0.943


In [96]:
from sklearn.metrics import precision_recall_fscore_support

correct = 0
y_train_pred = []
for i , p in enumerate(probs_train):
    y_train_pred.append(np.argmax(p))
    if np.argmax(p) == y_train[i]:
        correct +=1       
print("train acc: " , correct / len(y_train))


correct = 0
y_val_pred = []

for i , p in enumerate(probs_val):
    y_val_pred.append(np.argmax(p))
    if np.argmax(p) == y_val[i]:
        correct +=1       
print("val acc: " , correct / len(y_val))

pr = precision_recall_fscore_support(y_val, y_val_pred, average='weighted')
pr

train acc:  0.36054444444444445
val acc:  0.943


(0.9435735960209247, 0.943, 0.9429835050765621, None)

In [ ]:
# Concatenate the train set and the validation set
#full_train_data = torch.utils.data.ConcatDataset([train_data, val_data])
#full_train_sampler = RandomSampler(full_train_data)
#full_train_dataloader = DataLoader(full_train_data, sampler=full_train_sampler, batch_size=32)

# Train the Bert Classifier on the entire training data
#set_seed(42)
#bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
#train(bert_classifier, full_train_dataloader, epochs=2)

In [ ]:
#test_data.sample(5)

In [ ]:
# Run `preprocessing_for_bert` on the test set
#print('Tokenizing data...')
#test_inputs, test_masks = preprocessing_for_bert(test_data.text)

# Create the DataLoader for our test set
#test_dataset = TensorDataset(test_inputs, test_masks)
#test_sampler = SequentialSampler(test_dataset)
#test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

In [ ]:
# Compute predicted probabilities on the test set
#probs = bert_predict(bert_classifier, test_dataloader)

# Get predictions from the probabilities
#threshold = 0.9
#preds = np.where(probs[:, 1] > threshold, 1, 0)

# Number of tweets predicted non-negative
#print("Number of tweets predicted non-negative: ", preds.sum())

In [ ]:
#output = test_data[preds==1]
#list(output.sample(20).text)